# Detect yellow taxi using Custom Vision API


## Instruction
- This notebook detects objects using Custom Vision API, returns the Custom Vision prediction result in json format, and counts the number of target objects.
- For all PNG images in a single folder directory.

## Parameters:
- folder: folder path.
- Prediction_Key: refer to customvision.ai/projects - Performance - Prediction URL - Prediction-Key.
- Content_Type: default set as 'application/octet-stream' for image file prediction.
- projectId: refer to customvision.ai/projects - projectId.
- prediction_probability_threshold: Above this prediction probability threshold, we decide to detect the object as a target.

## Reference: 
* CustomVision documents: https://southcentralus.dev.cognitive.microsoft.com/docs/services/450e4ba4d72542e889d93fd7b8e960de/operations/5a6264bc40d86a0ef8b2c290


In [1]:
import pandas as pd
import numpy as np
import os, json
import http.client, urllib.request, urllib.parse, urllib.error, base64

In [2]:
# set parameters
folder = "/home/junjiecai/argo_groundwork/ETL_scripts/Images_sample"
Prediction_Key = '9c9942a1bc5e4226a28101bc38e87d1d'
Content_Type = 'application/octet-stream'
projectId = '51f1e733-1a74-4c08-85e9-19851feda644'
prediction_probability_threshold = 0.5

In [3]:
#This function detects objects using Custom Vision API, return the Custom Vision prediction result in json format.
def Custom_Vision_Prediction(img_file, Prediction_Key, Content_Type, projectId):    
    headers = {
        # Request headers
        'Prediction-Key': Prediction_Key,
        'Content-Type': Content_Type,
        'Prediction-key': '{subscription key}',
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'iterationId': '{}',
        'application': '{}',
    })

    try:
        with open(img_file, 'rb') as img:
            conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
            conn.request("POST", "/customvision/v2.0/Prediction/" + projectId + "/image?%s" % params, img, headers)
            response = conn.getresponse()
            data = response.read()
            return json.loads(data)
            conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [4]:
#This function reads the Custom Vision prediction result and counts the number of target objects in a single image.
def object_count(data, prediction_probability_threshold):
    num = 0
    for pred in data['predictions']:    
        prob = pred['probability']
        if prob > prediction_probability_threshold:
            num += 1
    return(num)

In [5]:
filenames = !ls $folder | grep .png

In [6]:
#set dataframe
index = [filenames]
columns = ['lon', 'lat', 'num']
data = np.array([np.zeros(len(filenames))]*3).T
df = pd.DataFrame(data, index=index, columns=columns)

In [7]:
%%time
for i in filenames:
    img_file = folder + '/' + i   
    
    data = Custom_Vision_Prediction(img_file, Prediction_Key, Content_Type, projectId)
    num = object_count(data, prediction_probability_threshold)
    
    df['lon'][i] = (i.split('_')[1])
    df['lat'][i] = (i.split('_')[0])
    df['num'][i] = (num)
df = df.reset_index().drop(columns=['level_0'])
df.to_csv('CustomVisionPrediction.csv')

CPU times: user 375 ms, sys: 78.1 ms, total: 453 ms
Wall time: 8 s


In [8]:
print(df.shape)
df.head()

(10, 3)


,lon,lat,num
0,-74.011550,40.704951,0.0
1,-74.011550,40.704951,0.0
2,-74.011550,40.704951,0.0
3,-74.014749,40.706344,0.0
4,-74.014749,40.706344,0.0
5,-73.736158,40.718218,1.0
6,-73.996572,40.729552,1.0
7,-73.923149,40.735353,1.0
8,-73.968024,40.751390,1.0
9,-73.968024,40.751390,1.0
